In [2]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
print("Libraries imported.")

Libraries imported.


To go [Part-2](#PART-2)  
To go [Part-3](#PART-3)

## Download Dataset 

To obtain the dataset which includes the **postal code, borough and neighborhood** of Toronto, we will utilize the Wikipedia page and by scraping the table in [this link](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M), we will be able to generate the required dataframe. 

Get the html of the wikipedia page with **requests** and **beautifulsoup** libraries.

In [3]:
# GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

We need 3 lists which includes the postal code, borough and neighborhood.

In [4]:
postalCode=[]
borough=[]
neighborhood=[]

The required information is in the "tr" and "td" HTML items. Former one shows the rows and latter one denotes the ingredient of that row.

In [5]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCode.append(cells[0].text.rstrip('\n'))
        borough.append(cells[1].text.rstrip('\n'))
        neighborhood.append(cells[2].text.rstrip('\n'))

Now we have 3 lists with the information. Next step, we will create dataframe for these.

In [6]:
print(len(postalCode))
print(len(borough))
print(len(neighborhood))

180
180
180


### Create Pandas Dataframe

In [13]:
toronto_df=pd.DataFrame({"Postal Code": postalCode,"Borough": borough,"Neighborhood": neighborhood})
toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**Instruction:** Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [14]:
toronto_df=toronto_df[toronto_df["Borough"] != "Not assigned"].reset_index(drop=True)
toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


**Instruction**: If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [15]:
toronto_df["Neighborhood"]=np.where(toronto_df["Neighborhood"]=="Not assigned",toronto_df["Borough"],toronto_df["Neighborhood"])

### Group by Borough and Postal Codes by putting "," between Neighborhoods. 

In [16]:
toronto_grouped = toronto_df.groupby(["Postal Code", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_grouped.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
toronto_grouped.shape

(103, 3)

# PART 2

Now, we will add to coordinates to the above dataframes by using Postal Codes. Here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [18]:
coor= pd.read_csv('https://cocl.us/Geospatial_data')
coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Merging 2 dataframes**

In [19]:
toronto_final = toronto_grouped.merge(coor, on="Postal Code", how="left")
toronto_final.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [22]:
toronto_final.shape

(103, 5)

# PART 3